In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = (11,7)
plt.style.use('seaborn')

# Load inflation rate dataset

In [ ]:
dt_inflation = pd.read_csv(r'../DataSet/inflation-rate-brazil.csv')
dt_inflation['date'] = pd.to_datetime(dt_inflation['date']).dt.to_period('M')
inflation_rate_dt = dt_inflation.set_index('date')['2011':'2019']
inflation_rate_dt.sample(n=5)

# Load gas price dataset

In [ ]:
original_gas_price_dt = pd.read_csv (r'..\\DataSet/2004-2021.tsv', sep = '\t')
is_gas = original_gas_price_dt['PRODUTO']=="GASOLINA COMUM"
df_gas = original_gas_price_dt[is_gas]

In [ ]:
df_gas['DATA FINAL'] = pd.to_datetime(df_gas['DATA FINAL'])

df_gas['PREÇO MÉDIO DISTRIBUIÇÃO'] = pd.to_numeric(df_gas['PREÇO MÉDIO DISTRIBUIÇÃO'], errors='coerce')
df_gas['PREÇO MÉDIO REVENDA'] = pd.to_numeric(df_gas['PREÇO MÉDIO REVENDA'], errors='coerce')

df_gas.set_index(['DATA FINAL'], inplace=True)
df_gas.head()

# Filtering gas dataset

In [ ]:
used_columns = [
    'DATA',
    'PRODUTO',
    'ESTADO',
    'REGIÃO',
    'PREÇO MÉDIO DISTRIBUIÇÃO',
    'PREÇO MÉDIO REVENDA',
]

monthly_df = df_gas.groupby(['PRODUTO', 'ESTADO', 'REGIÃO']).resample('M').mean().reset_index()
monthly_df['DATA'] = monthly_df['DATA FINAL'].dt.to_period('M')

gas_prices_df = monthly_df[used_columns]
gas_prices_df = gas_prices_df.dropna(how='any')

gas_prices_df['VARIAÇÃO DIST X REV'] = gas_prices_df['PREÇO MÉDIO REVENDA'] - gas_prices_df['PREÇO MÉDIO DISTRIBUIÇÃO']
gas_prices_df['VARIAÇÃO PERCENTUAL DIST X REV'] = gas_prices_df['VARIAÇÃO DIST X REV'] /  gas_prices_df['PREÇO MÉDIO DISTRIBUIÇÃO'] * 100

gas_prices_df.set_index(['DATA', 'PRODUTO', 'ESTADO', 'REGIÃO'], inplace=True)
gas_prices_df.sample(n=5)

# Treating inflation data

In [ ]:
inflation_rate_gas_price_dt = gas_prices_df.copy()
inflation_rate_gas_price_dt.sample(5)
inflation_rate_gas_price_dt.reset_index(inplace=True)
inflation_rate_gas_price_dt.set_index('DATA', inplace=True)
inflation_rate_gas_price_dt['INFLAÇÃO ANUAL'] = inflation_rate_dt['annual_accumulation']
inflation_rate_gas_price_dt['INFLAÇÃO ABSOLUTA'] = inflation_rate_dt['absolute_index']

inflation_rate_gas_price_dt = inflation_rate_gas_price_dt.to_timestamp()

inflation_rate_gas_price_dt = inflation_rate_gas_price_dt[:'2019']
inflation_rate_gas_price_dt.sample(5)

# Comparing inflation effect on gas price

In [ ]:
# TODO: Limpar datas que não estão sendo utilizadas do dt

annual_price_change_df = inflation_rate_gas_price_dt.groupby(['DATA']).mean()
annual_price_change_df['VARIAÇÃO'] = annual_price_change_df['PREÇO MÉDIO REVENDA'].pct_change()
annual_price_change_df['VARIAÇÃO 12 MESES'] = annual_price_change_df['VARIAÇÃO'].rolling(min_periods=12, window=12).sum() * 100
annual_price_change_df.tail()

# ploting
fig, ax_gas = plt.subplots()
annual_price_change_df
# annual_price_change_df.plot(y='VARIAÇÃO 12 MESES', c='#4c72b0', ax=ax_gas)
# fig.suptitle('Variação da Inflação e Preço Acumulado - ' + 'GASOLINA COMUM')
# ax_gas.set_xlabel('Data de pesquisa')
# ax_gas.set_ylabel('% - Variação Acumulativa - 12 meses', color='#4c72b0')
# ax_gas.get_legend().remove()
# ax_gas.grid(True)
#
# # plot_inflation
# ax_inflation = ax_gas.twinx()
#
# annual_price_change_df.plot(y='INFLAÇÃO ANUAL', ax=ax_inflation, c='#55a868')
# ax_inflation.set_ylabel('% - Inflação Acumulada - 12 meses', color='#55a868')
# ax_inflation.get_legend().remove()
# ax_inflation.grid(False)
# plt.show()

In [ ]:
def plot_avg_region():
    avg_per_region_for_product_df = inflation_rate_gas_price_dt['2011':].groupby(['PRODUTO', 'REGIÃO', 'DATA']).mean()

    fig, ax = plt.subplots()
    fig.suptitle('Preço por Região - ' + 'GASOLINA COMUM')

    for key, grp in avg_per_region_for_product_df.groupby('REGIÃO'):
        grp_as_timeseries = grp.reset_index().set_index('DATA')
        grp_as_timeseries.plot(y='PREÇO MÉDIO REVENDA', label=key, ax=ax)

        ax.set_xlabel('Data')
        ax.set_ylabel('Preço - R$/l')

    plt.grid(True)
    plt.show()

plot_avg_region()